<a href="https://colab.research.google.com/github/abirhazra/Reinforcement-Learning/blob/main/Deep_Reinforcement_Learning(gym_environment).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reference: Nicholas Renotte

#Steps for Deep Reinforcement Learning
1) Creating environments from OpenAi 

2) Building a tf.keras DL model

3) Using keras-rl agents to train

#Step 1. Install Dependencies

In [78]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

In [79]:
import gym 
import random

In [84]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [85]:
states

4

In [86]:
actions

2

In [88]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:34.0
Episode:2 Score:41.0
Episode:3 Score:12.0
Episode:4 Score:18.0
Episode:5 Score:19.0
Episode:6 Score:37.0
Episode:7 Score:10.0
Episode:8 Score:13.0
Episode:9 Score:32.0
Episode:10 Score:27.0


In [89]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [90]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [92]:
model = build_model(states, actions)

In [93]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 4)                 0         
_________________________________________________________________
dense_47 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_48 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_49 (Dense)             (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [94]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [96]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)

In [ ]:
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.text(env, nb_episodes=100, visualize = False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_actions=15, visualize = True)

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)